In [1]:
# Imports

import pandas as pd
import lightkurve as lk

/Users/swebber/pythonvenvs/quantumcomet/lib/python3.10/site-packages/lightkurve/prf/__init__.py:7: UserWarning: Warning: the tpfmodel submodule is not available without oktopus installed, which requires a current version of autograd. See #1452 for details.
  warnings.warn(


In [2]:
# load in the pandas dataframe
df = pd.read_csv("KeplerObjectsOfInterest.csv", comment="#")

df


,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,10811496,K00753.01,NaN,CANDIDATE,CANDIDATE,0.000,0,0,0,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9559,10090151,K07985.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,1,0,...,-166.0,4.529,0.035,-0.196,0.903,0.237,-0.079,297.18875,47.093819,14.082
9560,10128825,K07986.01,NaN,CANDIDATE,CANDIDATE,0.497,0,0,0,0,...,-220.0,4.444,0.056,-0.224,1.031,0.341,-0.114,286.50937,47.163219,14.757
9561,10147276,K07987.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.021,0,0,1,0,...,-236.0,4.447,0.056,-0.224,1.041,0.341,-0.114,294.16489,47.176281,15.385
9562,10155286,K07988.01,NaN,CANDIDATE,CANDIDATE,0.092,0,0,0,0,...,-128.0,2.992,0.030,-0.027,7.824,0.223,-1.896,296.76288,47.145142,10.998


In [3]:
# kepid for (koi_disposition == CONFIRMED && koi_pdisposition == CANDIDATE) or (koi_disposition == FALSE POSITIVE && koi_pdisposition == FALSE POSITIVE)
# kepid_confirmed = df.loc[(df['koi_disposition'] == "CONFRIMED") & (df['koi_pdisposition'] == 'CANDIDATE'), 'kepid']
kepid_confirmed = df[df['koi_disposition'] == 'CONFIRMED'][df['koi_pdisposition'] == 'CANDIDATE']['kepid']

confirmed_stars_to_planets = {}
for id in kepid_confirmed:
    confirmed_stars_to_planets[id] = df[df['kepid'] == id]['kepler_name'].tolist()

kepid_false_pos = df[df['koi_disposition'] == 'FALSE POSITIVE'][df['koi_pdisposition'] == 'FALSE POSITIVE']['kepid']

fp_stars_to_planets = {}
for id in kepid_false_pos:
    fp_stars_to_planets[id] = df[df['kepid'] == id]['kepler_name'].tolist()



/var/folders/d4/hhnp4w754y1446g43_srytjh0000gn/T/ipykernel_49343/63737405.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  kepid_confirmed = df[df['koi_disposition'] == 'CONFIRMED'][df['koi_pdisposition'] == 'CANDIDATE']['kepid']
/var/folders/d4/hhnp4w754y1446g43_srytjh0000gn/T/ipykernel_49343/63737405.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  kepid_false_pos = df[df['koi_disposition'] == 'FALSE POSITIVE'][df['koi_pdisposition'] == 'FALSE POSITIVE']['kepid']


In [4]:
for id, planets in confirmed_stars_to_planets.items():
    print(id)
    print(planets)
    break

10797460
['Kepler-227 b', 'Kepler-227 c']


In [6]:
import pickle
import os

# Create directories to store curves
os.makedirs("./confirmed_curves", exist_ok=True)
os.makedirs("./false_positive_curves", exist_ok=True)

In [ ]:
confirmed_curves = {} # Maps kepid of confirmed candidates to the corresponding light curve
curve_count = len(confirmed_stars_to_planets)

for i, (id, planets) in enumerate(confirmed_stars_to_planets.items()):

    filepath = f"confirmed_curves/confirmed_{id}.pkl"
    try:
        if not os.path.exists(filepath):
            print(f"Working on {i}/{curve_count}")
            lcf_collection = lk.search_lightcurve(f'KIC {id}', mission='Kepler').download_all()
            lc_stitched = lcf_collection.stitch()

            data_to_save = {
                "planets" : planets,
                "time": lc_stitched.time.value,
                "flux": lc_stitched.flux.value,
                "flux_err": lc_stitched.flux_err.value
            }
            confirmed_curves[id] = data_to_save

            with open(filepath, 'wb') as f:
                pickle.dump(data_to_save, f)
        else:
            print(f"Passing on {i}/{curve_count}")
    except Exception as e:
        print("************************************************************************************")
        print(e)
        print("************************************************************************************")
        continue






Passing on 0/1967
Working on 1/1967
Working on 2/1967
Working on 3/1967


*************************
Error in reading Data product /Users/swebber/.lightkurve/cache/mastDownload/Kepler/kplr010910878_lc_Q011111110111011101/kplr010910878-2013011073258_llc.fits of type KeplerLightCurve .
This file may be corrupt due to an interrupted download. Please remove it from your disk and try again.
*************************
Working on 4/1967


KeyboardInterrupt: 

In [8]:
fp_curves = {} # Maps kepid of confirmed candidates to the corresponding light curve
curve_count = len(fp_stars_to_planets)

for i, (id, planets) in enumerate(fp_stars_to_planets.items()):

    filepath = f"false_positive_curves/fp_{id}.pkl"
    try:
        if not os.path.exists(filepath):
            print(f"Working on {i}/{curve_count}")
            lcf_collection = lk.search_lightcurve(f'KIC {id}', mission='Kepler').download_all()
            lc_stitched = lcf_collection.stitch()

            data_to_save = {
                "planets" : planets,
                "time": lc_stitched.time.value,
                "flux": lc_stitched.flux.value,
                "flux_err": lc_stitched.flux_err.value
            }
            fp_curves[id] = data_to_save

            with open(filepath, 'wb') as f:
                pickle.dump(data_to_save, f)
        else:
            print(f"Passing on {i}/{curve_count}")
    except Exception as e:
        print("************************************************************************************")
        print(e)
        print("************************************************************************************")
        continue


Working on 0/4712
Working on 1/4712
Working on 2/4712


KeyboardInterrupt: 

In [ ]:
for id, result in confirmed_curves.items():
    print(type(result))
    print(result)
    

<class 'lightkurve.lightcurve.KeplerLightCurve'>
       time             flux      ...   pos_corr1      pos_corr2   
                                  ...      pix            pix      
------------------ -------------- ... -------------- --------------
 131.5120633522456  1.0023584e+00 ... -9.2174588e-03 -1.3527647e-02
131.53249777646124  1.0027753e+00 ... -9.1735851e-03 -1.3551758e-02
131.55293210056698  1.0018084e+00 ... -9.3217976e-03 -1.3630440e-02
131.57336632466468  1.0032423e+00 ... -9.3374131e-03 -1.3619965e-02
 131.5938006487704  1.0020605e+00 ... -9.3002012e-03 -1.3183980e-02
 131.6142350727605  1.0027404e+00 ... -9.1944840e-03 -1.3219508e-02
131.63466929686547  1.0031055e+00 ... -9.1631282e-03 -1.3154156e-02
 131.6551036207311  1.0023866e+00 ... -9.0096751e-03 -1.3044867e-02
131.67553804472118  1.0023539e+00 ... -9.0877246e-03 -1.3034637e-02
               ...            ... ...            ...            ...
1590.8173751198192  9.9907482e-01 ... -7.9931421e-03 -1.2251953e-02